In [37]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import scipy.io
import time


## Load data

In [26]:
mat = scipy.io.loadmat('linear_svm.mat')
X_train = mat["X_train"]
X_test = mat["X_test"]
y_train = mat["labels_train"]
y_test = mat["labels_test"]

## Define SVM problem and solve it with CVXOPT

In [28]:
N,M = X_train.shape
C = 5
P = np.zeros([N+1+M, N+1+M])
for i in range(M):
    P[i,i] = 0.5

q = np.zeros([N+1+M,])
for i in range(1,N+1):
    q[-i] = C

G = np.zeros([2 * N, N + 1 + M])
for i in range(N):
    G[i, :M] = y_train[i] * X_train[i]
    G[i, M] = y_train[i]
    G[i, M + 1 + i] = 1
    G[N + i, M + 1 + i] = 1
G = - G 

h = np.concatenate((- np.ones([N,]), np.zeros([N,])))

In [ ]:
cvxopt_solvers.options['reltol'] = 1e-5 # stopping condition

P = cvxopt_matrix(P)
q = cvxopt_matrix(q)
G = cvxopt_matrix(G)
h = cvxopt_matrix(h)

t1 = time.time()
sol = cvxopt_solvers.qp(P, q, G, h)
t2 = time.time()
x = np.array(sol['x'])

In [39]:
# Extract parameters w and b
w = x[:M]
b = x[M]

## Evaluate model
On test set

In [ ]:
print("************ Accuracy *****************")
(np.sign(np.matmul(X_test, w) + b) == y_test).sum() / y_test.shape[0]

In [ ]:
print(f"Time for execution: {t2-t1}") 

In [32]:
# Save parameters
np.save("w_OtSP", w)
np.save("b_OtSP", b)